In [7]:
import pandas as pd
import graph as helper
import networkx as nx
import itertools

AttributeError: module 'networkx' has no attribute 'utils'

In [4]:
def compute_degrees(tsv_file):
    # build new data frame for degrees: 
    df = pd.read_csv(tsv_file, sep='\t')
    num_of_nodes = max(df.max())
    data = {'positive_in_degree': [0], 'positive_out_degree':[0], 'negative_in_degree': [0], 'negative_out_degree': [0]}
    degrees_df = pd.DataFrame(data, index = [node_id for node_id in range(num_of_nodes)])
    
    # fill new data frame with data:
    for node_id in range(num_of_nodes):
        positive_in = df[(df.ToNodeId == node_id) & (df.Sign == 1)].shape[0]
        positive_out = df[(df.FromNodeId == node_id) & (df.Sign == 1)].shape[0]
        negative_in = df[(df.ToNodeId == node_id) & (df.Sign == -1)].shape[0]
        negative_out = df[(df.FromNodeId == node_id) & (df.Sign == -1)].shape[0]
            
        degrees_df.at[node_id,'positive_in_degree'] = positive_in
        degrees_df.at[node_id, 'positive_out_degree'] = positive_out
        degrees_df.at[node_id, 'negative_in_degree'] = negative_in
        degrees_df.at[node_id, 'negative_out_degree'] = negative_out
    
    return degrees_df

## TODO: USE THE GRAPH DIRECTLY AND NOT NUM OF NODES!! ## 

In [5]:
# Each triad type is represented by four digits as each digit may be 0 or 1. 
# First dif
TRIADS_TYPES = [(0,0,0,0),
               (0,0,0,1),
               (0,0,1,0), 
               (0,0,1,1),
               (0,1,0,0),
               (0,1,0,1),
               (0,1,1,0),
               (0,1,1,1),
               (1,0,0,0),
               (1,0,0,1),
               (1,0,1,0),
               (1,0,1,1),
               (1,1,0,0),
               (1,1,0,1),
               (1,1,1,0),
               (1,1,1,1)]

def triad_to_number(triad):
    return TRIADS_TYPES.index(triad)

def get_triad_status(u, w, v, triad, graph_df):
    first_edge = 0
    second_edge = 0
    
    if (not triad[0]):
        first_edge = graph_df[(graph_df.ToNodeId == w) & (graph_df.FromNodeId == u) & (graph_df.Sign == -1 if triad[1] else 1)].shape[0]
    else:
        first_edge = graph_df[(graph_df.ToNodeId == u) & (graph_df.FromNodeId == w) & (graph_df.Sign == -1 if triad[1] else 1)].shape[0]
        
    if (not triad[2]):
        second_edge = graph_df[(graph_df.ToNodeId == w) & (graph_df.FromNodeId == v) & (graph_df.Sign == -1 if triad[3] else 1)].shape[0]
    else:
        second_edge = graph_df[(graph_df.ToNodeId == v) & (graph_df.FromNodeId == w) & (graph_df.Sign == -1 if triad[3] else 1)].shape[0]
        
    return 1 if (first_edge and second_edge) else 0
          
def compute_triads(tsv_file):
    graph_df = pd.read_csv(tsv_file, sep='\t')
    graph = helper.build_graph(tsv_file)
    undirected_graph = graph.to_undirected()
    
    triads_data = {str(triad) : [0] for triad in TRIADS_TYPES}
    triads_index = [(u,v) for (u,v) in itertools.permutations(graph_df.nodes(), 2)]
    triads_df = pd.DataFrame(triads_data, triads_index)
    
    for (u,v) in itertools.permutations(graph.nodes(), 2):
        for w in sorted(nx.common_neighbors(undirected_graph, u, v)):
            for triad in TRIADS_TYPES:
                triads_df.at[(u,v), triad] = triads_df.at[(u,v), triad] + get_triad_status(u, w, v, triad, graph_df)

    return triads_df

In [2]:
triads_df = compute_triads("./datasets/wiki.tsv")

In [5]:
triads_df

,"(0, 0, 0, 0)","(0, 0, 0, 1)","(0, 0, 1, 0)","(0, 0, 1, 1)","(0, 1, 0, 0)","(0, 1, 0, 1)","(0, 1, 1, 0)","(0, 1, 1, 1)","(1, 0, 0, 0)","(1, 0, 0, 1)","(1, 0, 1, 0)","(1, 0, 1, 1)","(1, 1, 0, 0)","(1, 1, 0, 1)","(1, 1, 1, 0)","(1, 1, 1, 1)"
"(3, 30)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"(3, 54)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"(3, 214)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"(3, 28)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"(3, 286)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(8274, 7637)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"(8274, 8270)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"(8274, 8272)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"(8274, 8273)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


1 is true


,FromNodeId,ToNodeId,Sign
0,3,30,1
1,3,54,1
2,3,214,1
3,3,28,1
4,3,286,1
...,...,...,...
103181,8272,4940,1
103182,8273,4940,1
103183,8150,8275,-1
103184,8150,8276,-1


In [55]:
graph_df[(graph_df.ToNodeId == 30) & (graph_df.FromNodeId == 3) & (graph_df.Sign == 1)].shape[0]

def get_triad_status(u, w, v, triad, graph_df):
    first_edge = 0
    second_edge = 0
    
    if (not triad[0]):
        first_edge = graph_df[(graph_df.ToNodeId == w) & (graph_df.FromNodeId == u) & (graph_df.Sign == -1 if triad[1] else 1)].shape[0]
    else:
        first_edge = graph_df[(graph_df.ToNodeId == u) & (graph_df.FromNodeId == w) & (graph_df.Sign == -1 if triad[1] else 1)].shape[0]
        
    if (not triad[2]):
        second_edge = graph_df[(graph_df.ToNodeId == w) & (graph_df.FromNodeId == v) & (graph_df.Sign == -1 if triad[3] else 1)].shape[0]
    else:
        second_edge = graph_df[(graph_df.ToNodeId == v) & (graph_df.FromNodeId == w) & (graph_df.Sign == -1 if triad[3] else 1)].shape[0]
        
    return 1 if (first_edge and second_edge) else 0
        


0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0


In [7]:
def get_triads(u, v):
    df[((df.FromNodeId == u) | (df.ToNodeId == u)) | (df.FromNodeId == v) | (df.ToNodeId == v)]


In [8]:
G = helper.build_graph('./datasets/soc-sign-epinions.tsv')

In [31]:
if(1&1):
    print("NO")

NO
